# Capstone Project: Books recommender system

### Overall Contents:
- Background
- Data Collection
- Data Cleaning Booklist
- [Data Cleaning Book Interactions](#4.-Data-Cleaning-Book-Interactions) **(In this notebook)**
- Exploratory Data Analysis
- Modeling 1 Popularity-based system
- Modeling 2 Content-based system
- Modeling 3 Collaborative-based system
- Evaluation
- Conclusion and Recommendation

### Datasets

The dataset are obtained from [University of California San Diego Book Graph](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home?authuser=0).

The dataset contains meta-data of books and user-book interactions.

Meta-data of books:-
* goodreads_books 
* goodreads_book_authors
* goodreads_book_series
* goodreads_book_genres_initial

User-book interactions:-
* goodreads_interactions
* book_id_map

For more details on the datasets, please refer to the data_dictionary.ipynb.

## 4. Data Cleaning Books Interactions

### 4.1 Libraries Import

In [1]:
import numpy as np
import pandas as pd

# Maximum display of columns
pd.options.display.max_colwidth = 2000
pd.options.display.max_rows = 2000

### 4.2 Data Import

In [2]:
booklist_series = pd.read_parquet("../data/booklist_series.parquet")
booklist_compiled = pd.read_parquet("../data/booklist_compiled_clean.parquet")
booklist_genres = pd.read_parquet("../data/booklist_genres.parquet")
booklist_interactions = pd.read_parquet("../data/booklist_interactions.parquet")
bookid_removed = pd.read_parquet("../data/bookid_removed.parquet")
book_id_map = pd.read_parquet("../data/book_id_map.parquet")

In [3]:
print(f"This booklist_series has a shape of {booklist_series.shape}")
print(f"This booklist_genres has a shape of {booklist_genres.shape}")
print(f"This booklist_interactions has a shape of {booklist_interactions.shape}")
print(f"This bookid_removed has a shape of {bookid_removed.shape}")
print(f"This book_id_map has a shape of {book_id_map.shape}")

This booklist_series has a shape of (400390, 7)
This booklist_genres has a shape of (2360655, 2)
This booklist_interactions has a shape of (228648342, 5)
This bookid_removed has a shape of (524, 1)
This book_id_map has a shape of (2360650, 2)


### 4.3 booklist_series

In [4]:
booklist_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400390 entries, 0 to 400389
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   numbered            400390 non-null  object
 1   note                400390 non-null  object
 2   description         400390 non-null  object
 3   title               400390 non-null  object
 4   series_works_count  400390 non-null  int64 
 5   series_id           400390 non-null  int64 
 6   primary_work_count  400390 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 21.4+ MB


In [5]:
booklist_series.numbered.unique()

array(['true', 'false'], dtype=object)

In [6]:
# Change the numbered column to numeric and select the columns
booklist_series.numbered = booklist_series.numbered.map({'true': 1, 'false': 0})
booklist_series = booklist_series.drop(["note", "description"], axis = 1)
booklist_series.head()

,numbered,title,series_works_count,series_id,primary_work_count
0,1,Sun Wolf and Starhawk,9,189911,3
1,1,Avalon: Web of Magic,14,151854,12
2,1,Crowner John Mystery,15,169353,15
3,1,Pluto's Snitch,3,1052227,3
4,0,"The Way, With The Enchanted Neighborhood",3,1070125,3


In [7]:
booklist_series.describe()

,numbered,series_works_count,series_id,primary_work_count
count,400390.000000,400390.000000,4.003900e+05,400390.000000
mean,0.889153,21.588149,6.230450e+05,19.771653
std,0.313943,65.103100,2.944453e+05,63.501377
min,0.000000,-14.000000,1.443920e+05,0.000000
25%,1.000000,3.000000,3.637372e+05,3.000000
50%,1.000000,6.000000,6.158370e+05,5.000000
75%,1.000000,14.000000,8.775648e+05,12.000000
max,1.000000,893.000000,1.143859e+06,893.000000


In [8]:
booklist_series[booklist_series.series_works_count<0]

,numbered,title,series_works_count,series_id,primary_work_count
28592,1,Brie,-14,829188,3
83487,1,Brie,-14,829190,3
265286,1,Brie,-14,829189,3


**Analysis: Presence of negative values for count, which suggests it is a typo.**

In [9]:
booklist_series.series_works_count = booklist_series.series_works_count.apply(lambda x:abs(x) if x<0 else x)
booklist_series.loc[[28592, 83487, 265286], :]

,numbered,title,series_works_count,series_id,primary_work_count
28592,1,Brie,14,829188,3
83487,1,Brie,14,829190,3
265286,1,Brie,14,829189,3


### 4.4 booklist_genres

In [10]:
booklist_genres.head()

,book_id,genres
0,5333265,"{'children': None, 'comics, graphic': None, 'fantasy, paranormal': None, 'fiction': None, 'history, historical fiction, biography': 1.0, 'mystery, thriller, crime': None, 'non-fiction': None, 'poetry': None, 'romance': None, 'young-adult': None}"
1,1333909,"{'children': None, 'comics, graphic': None, 'fantasy, paranormal': None, 'fiction': 219.0, 'history, historical fiction, biography': 5.0, 'mystery, thriller, crime': None, 'non-fiction': None, 'poetry': None, 'romance': None, 'young-adult': None}"
2,7327624,"{'children': None, 'comics, graphic': None, 'fantasy, paranormal': 31.0, 'fiction': 8.0, 'history, historical fiction, biography': None, 'mystery, thriller, crime': 1.0, 'non-fiction': None, 'poetry': 1.0, 'romance': None, 'young-adult': None}"
3,6066819,"{'children': None, 'comics, graphic': None, 'fantasy, paranormal': None, 'fiction': 555.0, 'history, historical fiction, biography': None, 'mystery, thriller, crime': 10.0, 'non-fiction': None, 'poetry': None, 'romance': 23.0, 'young-adult': None}"
4,287140,"{'children': None, 'comics, graphic': None, 'fantasy, paranormal': None, 'fiction': None, 'history, historical fiction, biography': None, 'mystery, thriller, crime': None, 'non-fiction': 3.0, 'poetry': None, 'romance': None, 'young-adult': None}"


In [11]:
booklist_genres.genres[0]

{'children': None,
 'comics, graphic': None,
 'fantasy, paranormal': None,
 'fiction': None,
 'history, historical fiction, biography': 1.0,
 'mystery, thriller, crime': None,
 'non-fiction': None,
 'poetry': None,
 'romance': None,
 'young-adult': None}

In [12]:
# Convert the list of genres into dataframe
booklist_genres = pd.DataFrame(list(booklist_genres.genres), index = booklist_genres.book_id)
booklist_genres.head()

,children,"comics, graphic","fantasy, paranormal",fiction,"history, historical fiction, biography","mystery, thriller, crime",non-fiction,poetry,romance,young-adult
book_id,,,,,,,,,,
5333265,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1333909,NaN,NaN,NaN,219.0,5.0,NaN,NaN,NaN,NaN,NaN
7327624,NaN,NaN,31.0,8.0,NaN,1.0,NaN,1.0,NaN,NaN
6066819,NaN,NaN,NaN,555.0,NaN,10.0,NaN,NaN,23.0,NaN
287140,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


In [13]:
# To check for missing values for all columns
booklist_genres[booklist_genres.isnull().all(axis=1)].head()

,children,"comics, graphic","fantasy, paranormal",fiction,"history, historical fiction, biography","mystery, thriller, crime",non-fiction,poetry,romance,young-adult
book_id,,,,,,,,,,
28575155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30227122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16037548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24994796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# To remove book_id that have missing values for all columns
booklist_genres = booklist_genres[booklist_genres.notnull().any(axis=1)]
print(f"This new booklist_genres has a shape of {booklist_genres.shape}")
booklist_genres.head()

This new booklist_genres has a shape of (1951142, 10)


,children,"comics, graphic","fantasy, paranormal",fiction,"history, historical fiction, biography","mystery, thriller, crime",non-fiction,poetry,romance,young-adult
book_id,,,,,,,,,,
5333265,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1333909,NaN,NaN,NaN,219.0,5.0,NaN,NaN,NaN,NaN,NaN
7327624,NaN,NaN,31.0,8.0,NaN,1.0,NaN,1.0,NaN,NaN
6066819,NaN,NaN,NaN,555.0,NaN,10.0,NaN,NaN,23.0,NaN
287140,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


### 4.5 booklist_interactions

In [15]:
print(f"Number of unique books in booklist_interactions are: {booklist_interactions.book_id.nunique()}")
booklist_interactions.head()

Number of unique books in booklist_interactions are: 2360650


,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


### 4.5.1 Remove book_id that user_id does not read and rate

In [16]:
booklist_interactions = booklist_interactions[booklist_interactions.rating != 0]
print(f"The new booklist_interactions has a shape of {booklist_interactions.shape}")
booklist_interactions.head()

The new booklist_interactions has a shape of (104551549, 5)


,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [17]:
print(f"The new booklist_interactions has a shape of {booklist_interactions.shape}")

The new booklist_interactions has a shape of (104551549, 5)


In [18]:
booklist_interactions.head()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [19]:
# To check for duplication in booklist_interactions
booklist_interactions[booklist_interactions.duplicated(subset=['user_id', 'book_id'])]

,user_id,book_id,is_read,rating,is_reviewed


In [20]:
booklist_interactions.dtypes

user_id        int64
book_id        int64
is_read        int64
rating         int64
is_reviewed    int64
dtype: object

### 4.5.2 Remove book_id that are removed from booklist_compiled

A list of books were removed from booklist_compiled due to the majority of the information of the books are missing including work_id, authors, average_rating and ratings_count.

Note: booklist_interactions uses an assigned book_id (book_id_csv) and any book_id from other dataframes including booklist_compiled need to be mapped to book_id_csv. book_id_map dataframe will be used as a reference to map the book_id and book_id_csv.

1. To remove the book_id removed from booklist_compiled
2. To map work_id to the book_id using information from booklist_compiled

In [21]:
book_id_map.head()

,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361


In [22]:
book_id_map.describe()

,book_id_csv,book_id
count,2.360650e+06,2.360650e+06
mean,1.180324e+06,1.540948e+07
std,6.814611e+05,1.081894e+07
min,0.000000e+00,1.000000e+00
25%,5.901622e+05,5.979043e+06
50%,1.180324e+06,1.585680e+07
75%,1.770487e+06,2.438191e+07
max,2.360649e+06,3.653043e+07


In [23]:
# To check the presence of book_id in booklist_interactions in book_id_map
booklist_interactions.book_id.isin(book_id_map.book_id_csv).astype(int).value_counts()

1    104551549
Name: book_id, dtype: int64

In [24]:
print(f'The number of book_id in book_id_map is {book_id_map.shape[0]}')
print(f'The number of book_id in booklist_interactions is {booklist_interactions.book_id.nunique()}')

The number of book_id in book_id_map is 2360650
The number of book_id in booklist_interactions is 2325541


**Analysis: book_id_map has all the book_id in booklist_interactions.**

**1. To remove the book_id removed from booklist_compiled**

In [25]:
# To map the book_id_csv with book_id in book_id_removed
bookid_removed = pd.merge(bookid_removed, book_id_map, on = 'book_id', how = 'left')
bookid_removed.head()

,book_id,book_id_csv
0,23699819,2279142
1,2597774,1913117
2,18521522,705086
3,28253116,2213971
4,17796597,1032105


In [26]:
# To indicate the presence of book_id in book_id removed
booklist_interactions["is_removed"] = booklist_interactions.book_id.isin(bookid_removed.book_id_csv).astype(int)
booklist_interactions.is_removed.value_counts()

0    104543831
1         7718
Name: is_removed, dtype: int64

In [27]:
# To remove the book_id present in book_id removed
booklist_interactions = booklist_interactions[booklist_interactions.is_removed == 0]
booklist_interactions = booklist_interactions.drop(["is_removed"], axis = 1)
booklist_interactions.head()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


**2. To map work_id to the book_id using information from booklist_compiled**

In [28]:
# Create a list of work_id in book_id_map
book_work_csv_id = booklist_compiled[["book_id", "work_id"]]
book_work_csv_id = pd.merge(book_work_csv_id, book_id_map, on = 'book_id', how = 'left')
book_work_csv_id.head()

,book_id,work_id,book_id_csv
0,5333265,5400751,1950356.0
1,1333909,1323437,2084644.0
2,7327624,8948723,740362.0
3,6066819,6243154,14854.0
4,287140,278577,979469.0


In [29]:
book_work_csv_id.isnull().value_counts()

book_id  work_id  book_id_csv
False    False    False          2360126
                  True                 5
dtype: int64

In [30]:
book_work_csv_id[book_work_csv_id.book_id_csv.isnull()]

,book_id,work_id,book_id_csv
166287,35230241,52573954,NaN
764326,36498955,45629111,NaN
1275442,35604756,56556476,NaN
1720805,36511533,209150,NaN
1991947,36517163,54759550,NaN


**Analysis: Presence of 5 book_id in booklist_compiled that are not present in book_work_csv.**

This observation is not critical as all book_id in booklist_interactions are present in book_id_map.

In [31]:
# To generate a dataframe only with work_id and book_id
bookid_comparison = book_work_csv_id.drop(["book_id"], axis = 1)
bookid_comparison = bookid_comparison.rename({"book_id_csv" : "book_id"}, axis = 1)
bookid_comparison.head()

,work_id,book_id
0,5400751,1950356.0
1,1323437,2084644.0
2,8948723,740362.0
3,6243154,14854.0
4,278577,979469.0


In [32]:
booklist_interactions = pd.merge(booklist_interactions,bookid_comparison, on = "book_id", how = 'left')
print(f'Whether there are missing values in booklist_interactions {booklist_interactions.work_id.isnull().value_counts()}')
booklist_interactions.head()

Whether there are missing values in booklist_interactions False    104543831
Name: work_id, dtype: int64


,user_id,book_id,is_read,rating,is_reviewed,work_id
0,0,948,1,5,0,135328
1,0,947,1,5,1,2305997
2,0,946,1,5,0,89369
3,0,945,1,5,0,1699340
4,0,944,1,5,0,41335427


In [33]:
# To verify whether are there any duplication for work_id for each user_id
booklist_interactions[booklist_interactions.duplicated(subset=['user_id', 'work_id'])].head()

,user_id,book_id,is_read,rating,is_reviewed,work_id
399,0,268,1,4,0,1177001
732,2,1096,1,3,0,1041558
1115,5,6495,1,3,0,19929609
1148,5,5851,1,3,0,4117634
1151,5,1604,1,5,0,15524542


In [34]:
booklist_interactions[(booklist_interactions.work_id == 1177001) & (booklist_interactions.user_id == 0)]

,user_id,book_id,is_read,rating,is_reviewed,work_id
83,0,847,1,5,1,1177001
399,0,268,1,4,0,1177001


In [35]:
user_work_interactions = booklist_interactions.groupby(['user_id','work_id'])['rating'].mean().reset_index()

In [36]:
user_work_interactions[(user_work_interactions.user_id == 0) & (user_work_interactions.work_id == 1177001)]

,user_id,work_id,rating
176,0,1177001,4.5


### 4.6 Summary

* Handled data values including changing to integers/floats and rename values for consistency
* Selected columns for booklist_series
* Converted the genre into columns and remove books without any genre in booklist_genres
* Remove books that are not read and without rating in booklist_interactions
* Remove books that are removed from booklist_compiled
* Map work_id to booklist_interactions and average the rating score.

## Exporting Data

In [38]:
#Placed the # to refrain from executing
booklist_genres.to_parquet("../data/booklist_genres_clean.parquet", compression = 'gzip') 
booklist_series.to_parquet("../data/booklist_series_clean.parquet", compression = 'gzip')
booklist_interactions.to_parquet("../data/booklist_interactions_clean.parquet", compression = 'gzip')
book_work_csv_id.to_parquet("../data/book_work_csv_id.parquet", compression = 'gzip')
user_work_interactions.to_parquet("../data/user_work_interactions.parquet", compression = 'gzip')